In [3]:
from robotic import ry
import numpy as np
import time

## Part a) Adding a constraint to simulation

In [4]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
C.addFrame('way1'). setShape(ry.ST.marker, [.1]) .setPosition([.4, .2, 1.])
C.addFrame('way3'). setShape(ry.ST.marker, [.1]) .setPosition([-.4, .2, 1.])

C.addFrame('box') \
    .setPosition([-.25,.1,1.]) \
    .setShape(ry.ST.ssBox, size=[.06,.6,.6,.005]) \
    .setColor([1,.5,0]) \
    .setContact(True)

C.view()
qHome = C.getJointState()

First, solve the task without collision constraint

In [6]:
C.setJointState(qHome)
komo = ry.KOMO(C, 5, 10, 1, True)
komo.addControlObjective([], 0, 1e-1)
komo.addControlObjective([], 1, 1e0)
komo.addObjective([1], ry.FS.positionDiff, ['l_gripper', 'way1'], ry.OT.eq, [1e1])
komo.addObjective([3], ry.FS.positionDiff, ['l_gripper', 'way3'], ry.OT.eq, [1e1])
komo.addObjective([5], ry.FS.positionDiff, ['l_gripper', 'way1'], ry.OT.eq, [1e1])

ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
print(ret)
q = komo.getPath()
print(q)

for t in range(q.shape[0]):
    C.setJointState(q[t])
    C.view(False, f'waypoint {t}')
    time.sleep(0.1)

{ time: 0.022578, evals: 15, done: 1, feasible: 1, sos: 1.88422, f: 0, ineq: 0, eq: 0.00134121 }
[[-3.70382074e-02 -4.54617309e-01 -3.78776378e-02 -2.00579252e+00
  -9.45349382e-03  2.01354200e+00 -4.99973172e-01]
 [-7.41172529e-02 -4.09185138e-01 -7.57916421e-02 -2.01158688e+00
  -1.89116215e-02  2.02709587e+00 -4.99946312e-01]
 [-1.11277927e-01 -3.63654163e-01 -1.13778433e-01 -2.01738506e+00
  -2.83791287e-02  2.04067340e+00 -4.99919385e-01]
 [-1.48560931e-01 -3.17975358e-01 -1.51874543e-01 -2.02318925e+00
  -3.78609833e-02  2.05428620e+00 -4.99892359e-01]
 [-1.86006841e-01 -2.72100118e-01 -1.90116674e-01 -2.02900194e+00
  -4.73624853e-02  2.06794561e+00 -4.99865203e-01]
 [-2.23656090e-01 -2.25980346e-01 -2.28541778e-01 -2.03482605e+00
  -5.68893727e-02  2.08166270e+00 -4.99837885e-01]
 [-2.61548962e-01 -1.79568497e-01 -2.67187129e-01 -2.04066491e+00
  -6.64479244e-02  2.09544820e+00 -4.99810376e-01]
 [-2.99725607e-01 -1.32817588e-01 -3.06090420e-01 -2.04652239e+00
  -7.60450562e-02 

Then, we add collision constraint with "accumulatedCollisions" feature

In [19]:
del komo
del C

In [20]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
C.addFrame('way1'). setShape(ry.ST.marker, [.1]) .setPosition([.4, .2, 1.])
C.addFrame('way3'). setShape(ry.ST.marker, [.1]) .setPosition([-.4, .2, 1.])

C.addFrame('box') \
    .setPosition([-.25,.1,1.]) \
    .setShape(ry.ST.ssBox, size=[.06,.6,.6,.005]) \
    .setColor([1,.5,0]) \
    .setContact(True)

C.view()
qHome = C.getJointState()

But we see that there is a jump in the motion while passing the obstacle

Here we will see the importance of the scale of the constraint (give values 0, 100 and 1 (which is default one) relatively). Then put 0.1 (1e-1) for perfect case. Do not forget to Del K and C or restart.

In [34]:
C.setJointState(qHome)
komo = ry.KOMO(C, 5, 10, 1, True)
komo.addControlObjective([], 0, 1e-1)
komo.addControlObjective([], 1, 1e0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq,[1e-1]);
komo.addObjective([1], ry.FS.positionDiff, ['l_gripper', 'way1'], ry.OT.eq, [1e1])
komo.addObjective([3], ry.FS.positionDiff, ['l_gripper', 'way3'], ry.OT.eq, [1e1])
komo.addObjective([5], ry.FS.positionDiff, ['l_gripper', 'way1'], ry.OT.eq, [1e1])

ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
print(ret)
q = komo.getPath()
print(q)

for t in range(q.shape[0]):
    C.setJointState(q[t])
    C.view(False, f'waypoint {t}')
    time.sleep(0.1)

{ time: 0.801347, evals: 267, done: 1, feasible: 1, sos: 4.014, f: 0, ineq: 0, eq: 0.0141815 }
[[-3.51565731e-02 -4.50905644e-01 -3.85410208e-02 -2.00015671e+00
  -1.30812804e-02  2.00507939e+00 -4.97746308e-01]
 [-7.03360150e-02 -4.01747876e-01 -7.71274853e-02 -2.00029244e+00
  -2.61913485e-02  2.01013218e+00 -4.95487446e-01]
 [-1.05561136e-01 -3.52463192e-01 -1.15804979e-01 -2.00038617e+00
  -3.93589496e-02  2.01513170e+00 -4.93218232e-01]
 [-1.40854632e-01 -3.02987914e-01 -1.54619370e-01 -2.00041675e+00
  -5.26127436e-02  2.02005116e+00 -4.90933454e-01]
 [-1.76239022e-01 -2.53258099e-01 -1.93616954e-01 -2.00036288e+00
  -6.59812595e-02  2.02486352e+00 -4.88627855e-01]
 [-2.11736592e-01 -2.03209458e-01 -2.32844601e-01 -2.00020305e+00
  -7.94928477e-02  2.02954153e+00 -4.86296114e-01]
 [-2.47369332e-01 -1.52777276e-01 -2.72349900e-01 -1.99991551e+00
  -9.31756289e-02  2.03405759e+00 -4.83932822e-01]
 [-2.83158879e-01 -1.01896340e-01 -3.12181311e-01 -1.99947820e+00
  -1.07057438e-01  2

komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq);

    komo.addObjective(...): This is calling a method named addObjective on the komo object, which is an instance of a KOMO optimizer.

    []: The first argument [] specifies the time intervals over which this objective applies. In this case, [] means it applies to all time intervals. In other words, this objective is valid for the entire duration of the optimization.

    ry.FS.accumulatedCollisions: The second argument ry.FS.accumulatedCollisions is specifying the type of feature to use for this objective. In this case, it's using a feature related to accumulated collisions. This means the objective will aim to minimize the total amount of collisions that occur throughout the motion.

    []: The third argument [] specifies the frames associated with this feature. Since ry.FS.accumulatedCollisions doesn't require specific frame associations, this argument is left empty.

    ry.OT.eq: The fourth argument ry.OT.eq specifies the type of objective. Here, it's an equality constraint. This means the optimizer will aim to make the value of the accumulated collisions equal to a certain target value (which is default value zero in here).

Putting it all together, this line is instructing the optimizer to minimize the total amount of accumulated collisions over the entire duration of the motion, treating it as an equality constraint. This is a way to prioritize collision-free or low-collision solutions.

##  Part b) Possible State Solution by KOMO

In [35]:
del komo
del C

In [42]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
C.addFrame('way3'). setShape(ry.ST.marker, [.1]) .setPosition([-.4, .2, 1.])

C.addFrame('box') \
    .setPosition([-.25,.1,1.3]) \
    .setShape(ry.ST.ssBox, size=[.06,.7,.5,.005]) \
    .setColor([1,.5,0]) \
    .setContact(True)

C.view()
qHome = C.getJointState()

In [38]:
C.setJointState(qHome)
komo = ry.KOMO(C, 1, 10, 2, True)
komo.addControlObjective([], 0, 1e-1) 
komo.addControlObjective([], 2, 1e0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq,[1e1]);
komo.addObjective([1], ry.FS.positionDiff, ['l_gripper', 'way3'], ry.OT.eq, [1e1])
komo.addObjective([1], ry.FS.jointState, [], ry.OT.eq, [1e1], [], order=1)

ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
print(ret)
q = komo.getPath()
print('size of path:', q.shape)

for t in range(q.shape[0]):
    C.setJointState(q[t])
    C.view(False, f'waypoint {t}')
    time.sleep(.1)

{ time: 0.279718, evals: 398, done: 1, feasible: 1, sos: 59.8335, f: 0, ineq: 0, eq: 0.0322768 }
size of path: (10, 7)


## Possible State Solution by RRT

In [43]:
# that's the goal configuration
qT = komo.getPath()[-1]
C.setJointState(qT)
C.view(False, "IK solution")

0

In [44]:
#define a path finding problem
rrt = ry.PathFinder()
rrt.setProblem(C, [qHome], [qT])

ret = rrt.solve()
print(ret)
path = ret.x

{ time: 0.011094, evals: 60, done: 1, feasible: 1, sos: -1, f: -1, ineq: -1, eq: -1 }


In [46]:
# display the path
for t in range(0, path.shape[0]-1):
    C.setJointState(path[t])
    C.view()
    time.sleep(.1)

## RRT Solution on BotOp 

In [47]:
# run the path with botop
C.setJointState(qHome)
ry.params_add({'botsim/verbose': 1., 'physx/motorKp': 10000., 'physx/motorKd': 1000.})
bot = ry.BotOp(C, False)
bot.home(C)

In [48]:
bot.moveAutoTimed(path, 1., 1.)
while bot.getTimeToEnd()>0:
    bot.sync(C, .1)

0.170185 1


In [49]:
del komo
del rrt
del bot
del C

-- bot.cpp:~BotOp:106(0) shutting down BotOp...
-- simulation.cpp:~BotThreadedSim:56(0) shutting down SimThread
-- simulation.cpp:~Simulation:148(0) shutting down Simulation


Lets give impossible state

##  Part c) Impossible State Solution by KOMO

In [50]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
C.addFrame('way3'). setShape(ry.ST.marker, [.1]) .setPosition([-.4, .2, 1.])

C.addFrame('box') \
    .setPosition([-.25,.1,1.]) \
    .setShape(ry.ST.ssBox, size=[.06,.7,.7,.005]) \
    .setColor([1,.5,0]) \
    .setContact(True)

C.view()
qHome = C.getJointState()

In [51]:
C.setJointState(qHome)
komo = ry.KOMO(C, 1, 10, 2, True)
komo.addControlObjective([], 0, 1e-1) 
komo.addControlObjective([], 2, 1e0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq,[1e-1]);
komo.addObjective([1], ry.FS.positionDiff, ['l_gripper', 'way3'], ry.OT.eq, [1e1])
komo.addObjective([1], ry.FS.jointState, [], ry.OT.eq, [1e1], [], order=1)

ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
print(ret)
q = komo.getPath()
print('size of path:', q.shape)

for t in range(q.shape[0]):
    C.setJointState(q[t])
    C.view(False, f'waypoint {t}')
    time.sleep(.1)

{ time: 0.057812, evals: 58, done: 1, feasible: 1, sos: 7.82926, f: 0, ineq: 0, eq: 0.176068 }
size of path: (10, 7)


## Impossible State Solution by RRT

In [52]:
# that's the goal configuration
qT = komo.getPath()[-1]
C.setJointState(qT)
C.view(False, "IK solution")

0

In [53]:
#define a path finding problem
rrt = ry.PathFinder()
rrt.setProblem(C, [qHome], [qT])

ret = rrt.solve()
print(ret)
path = ret.x

-- RRT_PathFinder.cpp:RRT_SingleTree:55(0) rooting RRT with infeasible start configuration -- that's likely to fail: query is:
query: h_goal: 0 g_coll: 0.191575 isGoal: 1 isFeasible: 0
{ time: 1.01741, evals: 5001, done: 1, feasible: 0, sos: -1, f: -1, ineq: -1, eq: -1 }


So we will get an error since there is no feasible solution

In [54]:
# display the path
for t in range(0, path.shape[0]-1):
    C.setJointState(path[t])
    C.view()
    time.sleep(.1)

IndexError: tuple index out of range